In [10]:
import pretty_midi as pm
# Env variables
chrom_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
offsets = {
    '1': 0,
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11,
    '8': 12,
    '9': 14,
    '10': 16,
    '11': 17,
    '12': 19,
    '13': 21
}

In [18]:
# Returns a note name based on its MIDI note number
def get_note(note_n):
    return chrom_notes[note_n % 12]

# Returns the correct note based on a root note of a scale and its degree
def parse_chord(root, number_string):
    note_num = chrom_notes.index(root)
    out = ""
    num = ""
    scale_num = 0
    parentheses = False
    for char in number_string:
        if char == '(':
            parentheses = True
        if char == 'b':
            scale_num -= 1
        if char == '#':
            scale_num += 1
        if char >= '0' and char <= '9':
            num += char
    scale_num += offsets.get(num)
    if (parentheses):
        out = "("
    out += str(get_note(note_num + scale_num))
    if (parentheses):
        out += ")"
    return out

# Outputs the master chord list (dict version)
def generate_chord_list(filepath = "chords without names.txt"):
    chord_list = []
    for note in chrom_notes:
        f = open(filepath)
        lines = f.readlines()
        for line in lines:
            parts = line.split()
            chord_name = ''
            note_list = []
            for i in range(len(parts)):
                part = parts[i]
                if i == 0:
                    chord_name = part.replace('_', note, 1)
                elif part[0] == 'b' or part[0] == '#' or \
                   (part[0] >= '0' and part[0] <= '9') or \
                   part[0] == '(':
                    note_list.append(parse_chord(note, part))
                else: continue
            chord_list.append([chord_name, note_list])
    return chord_list

# Gets the chords in a song
def get_chords(notes, 
               offset = 0.01):
    start_times = []
    for note in notes:
        if not (note.start in start_times):
            start_times.append(note.start)
    chords = []
    for time in start_times:
        playing_notes = []
        actual = time + offset
        for note in notes:
            if note.start < actual and note.end >= time:
                song.instruments[0].notes
                playing_notes.append(note)
        chords.append(playing_notes)
    return chords

# Generates note scores for the piece
def get_note_scores(notes, 
                    octave_multiplier_on = False,
                    end_multiplier_on = False,
                    printVals = False):
    note_scores_octave_agn = []
    note_scores_octave_agn_dict = dict()
    if end_multiplier_on:
        last_end = 0.0
        for note2 in notes:
            if note2.end > last_end:
                last_end = note2.end
    for i in range(0, 12):
        note_scores_octave_agn.append(0)
    for note in notes:
        duration = note.end - note.start
        score = duration * note.velocity / 127
        if octave_multiplier_on:
            octave_multiplier = max(0, 1 - (max(0, (round(note.pitch % 12) - 3) / 8.0)))
            print("%d %f" % (note.pitch, octave_multiplier))
        else:
            octave_multiplier = 1
        if end_multiplier_on:
            end_multiplier = note.end / last_end
        else:
            end_multiplier = 1
        score *= octave_multiplier
        score *= end_multiplier
        note_scores_octave_agn[note.pitch % 12] += score
    for i in range(0, 12):
        if note_scores_octave_agn[i] != 0:
            note_scores_octave_agn_dict[i] = note_scores_octave_agn[i]
    if printVals:
        note_scores_octave_agn_sorted = sorted(note_scores_octave_agn_dict.items(), key=lambda x:x[1], reverse = True)
        print(inst.name)
        for key in note_scores_octave_agn_dict.keys():
            print("Note %s has a score of %f" % (get_note(key), note_scores_octave_agn_dict[key]))
        print("The most common note is %s" % get_note(note_scores_octave_agn_sorted[0][0]))
        print("------")
        print()
    return note_scores_octave_agn_dict
    
# Generates chord scores based on note scores
def get_chord_scores(chord_list, 
                     note_scores_octave_agn_dict, 
                     parentheses_multiplier = 1
                     min_note_threshold = 0.1, 
                     missing_deweight = 0.5, 
                     printVals = False):
    chord_scores_dict = {}
    for chord_tuple in chord_list:
        chord_name = chord_tuple[0]
        chord_notes = chord_tuple[1]
        chord_score = 0.0
        for note in chord_notes:
            multiplier = 1
            actual_note = note
            if note[0] == '(':
                multiplier = parentheses_multiplier
                actual_note = note[1 : (len(note) - 1)]
            note_val = chrom_notes.index(actual_note)
            note_score = note_scores_octave_agn_dict.get(note_val, 0)
            if note_score <= min_note_threshold:
                note_score = -1 * missing_deweight # Deweight chords with missing notes
            chord_score += note_score * multiplier
        if chord_score <= 0.0:
            chord_scores_dict[chord_name] = chord_score
    chord_scores_dict_sorted = sorted(chord_scores_dict.items(), key=lambda x:x[1], reverse = True)
    if printVals:
        print("The 10 highest-scoring chords are:")
        for i in range(10):
            print("%d: %s with a score of %f" % ((i + 1), chord_scores_dict_sorted[i][0], chord_scores_dict_sorted[i][1]))
    return chord_scores_dict_sorted

In [12]:
song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\progression.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\Untitled score.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\7-Strings Ensemble Staccato 7 test.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\jazz but without the piano chords.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\blind comp render E-PIANO ONLY_basic_pitch.mid")

In [19]:
get_chord_scores(generate_chord_list(), get_note_scores(song.instruments[0].notes))[:10]

[('AbM7b6', 25.181102362204722),
 ('CM7b6', 23.292650918635168),
 ('FM7b6', 22.03280839895013),
 ('Bb7#11', 22.032808398950127),
 ('EbM7b6', 20.7742782152231),
 ('CM13', 20.774278215223095),
 ('Dm13', 20.774278215223095),
 ('FM#4', 20.774278215223095),
 ('FM13', 20.773622047244093),
 ('Gm13', 20.773622047244093)]

In [34]:
all_chords = generate_chord_list()
for chord in get_chords(song.instruments[0].notes):
    print(get_chord_scores(all_chords, get_note_scores(chord))[:1])

[('Fm/M7', 6.295931758530184)]
[('Em/M7', 4.5367454068241475)]
[('Dbsus2', 5.0367454068241475)]
[('Gm7', 2.5170603674540684)]
[('C7', 3.1463254593175853)]
[('FM9', 6.295931758530181)]
